### Imports

In [25]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import OneRClassifier
import time

### Loading and preprocessing images

In [ ]:
dataset_path = "Cat_Breed"
unseen_dataset_path = "Unseen_Cat_Breed"

breeds = {
    'Abyssinian': 0, 'Bombay': 1, 'Egyptian Mau': 2, 'Exotic Shorthair': 3,
    'Himalayan': 4, 'Maine Coon': 5, 'Regdoll': 6, 'Russian Blue': 7,
    'Scottish Fold': 8, 'Siamese': 9, 'Sphynx': 10
}

img_size = (64, 64)

def load_and_preprocess_images(directory, label, img_size):
    images, labels = [], []
    if not os.path.exists(directory):
        print(f"Warning: {directory} does not exist. Skipping...")
        return images, labels
    
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)      
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = img / 255.0
            images.append(img.reshape(64, 64, 1))
            labels.append(label)
    return images, labels

X, y = [], []
for breed, label in breeds.items():
    images, labels = load_and_preprocess_images(os.path.join(dataset_path, breed), label, img_size)
    X.extend(images)
    y.extend(labels)

X, y = np.array(X), np.array(y)

print(f"Loaded {len(X)} training images from {len(breeds)} classes.")

Loaded 4950 training images from 11 classes.


### Splitting Train and Test Images

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Training of Models

In [28]:
decision_tree_classifier = DecisionTreeClassifier()
naive_bayes_classifier = GaussianNB()
svm_classifier = SVC(probability=True)
random_forest_classifier = RandomForestClassifier(n_estimators=100)

In [29]:
def train_model(model, name):
    start_time = time.time() 
    model.fit(X_train, y_train)
    end_time = time.time() 
    training_time = end_time - start_time 
    print(f"{name} trained in {training_time:.4f} seconds.")


In [30]:
train_model(decision_tree_classifier, "Decision Tree")

Decision Tree trained in 14.5164 seconds.


In [31]:
train_model(naive_bayes_classifier, "Naive Bayes")

Naive Bayes trained in 0.1355 seconds.


In [32]:
train_model(svm_classifier, "SVM Classifier")

SVM Classifier trained in 91.6114 seconds.


In [33]:
train_model(random_forest_classifier, "Random Forest Classifier")

Random Forest Classifier trained in 15.6596 seconds.


### Saving the Model

In [37]:
import joblib

model_folder = 'saved_models'
os.makedirs(model_folder, exist_ok=True) 

joblib.dump(decision_tree_classifier, os.path.join(model_folder, "decision_tree.pkl"))
joblib.dump(naive_bayes_classifier, os.path.join(model_folder, "naive_bayes.pkl"))
joblib.dump(svm_classifier, os.path.join(model_folder, "svm_model.pkl"))
joblib.dump(random_forest_classifier, os.path.join(model_folder, "random_forest.pkl"))

['saved_models\\random_forest.pkl']

### Model Metrics

In [35]:
def check_model_metrics(model, name):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test) if hasattr(model, "predict_proba") else None

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="weighted", zero_division=1)
    recall = recall_score(y_test, y_pred, average="weighted", zero_division=1)
    roc_auc = roc_auc_score(y_test, y_prob, multi_class="ovr") if y_prob is not None else None
    cm = confusion_matrix(y_test, y_pred)

    cross_val = cross_val_score(model, X_train, y_train, cv=10).mean()

    print(f"{name} Metrics")
    print(f"Confirmation Metrix: {cm}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"ROC-AUC: {roc_auc}")

check_model_metrics(decision_tree_classifier, "Decision Tree")
check_model_metrics(naive_bayes_classifier, "Naive Bayes")
check_model_metrics(svm_classifier, "SVM Classifier")
check_model_metrics(random_forest_classifier, "Random Forest Classifier")
    


Decision Tree Metrics
Confirmation Metrix: [[25  4  8 11  6  7  1 11  6 10  8]
 [ 7 60  4  2  3  6  0  6  0  6  4]
 [18  2 11  7 12  5  2  7  5 10 11]
 [ 9  2 11 21  7  6 14  3 12 10  6]
 [ 5  0  8  5 22  5  6  6  3 12  5]
 [ 9  2 11  6  5 20  8  9 12  7  6]
 [ 2  1  4  9  7 10 18  4  9  7  7]
 [12  2 11  4 10  7  3 19  5  9 10]
 [ 3  1 12 10  3 10  8  8 24  9  6]
 [ 0 10 12  3 11 11  2  3  6 15 10]
 [ 5  0  3 11 10  6  4 11  7  5 23]]
Accuracy: 0.2606060606060606
Precision: 0.27008318100122797
Recall: 0.2606060606060606
ROC-AUC: 0.5926077627467404
Naive Bayes Metrics
Confirmation Metrix: [[38  4  1  2  0  2  3 31  6  4  6]
 [ 1 84  0  3  0  0  0  4  0  6  0]
 [28  2  2  2  1  8 10 18  1  5 13]
 [21  4  3  2  2 14 24 12 10  2  7]
 [16  4  3  3  6  9 11  7  3  4 11]
 [17  8  0  3  0 19 18 17  2  3  8]
 [ 5  0  1  1  1 16 46  1  1  0  6]
 [25  7  1  2  1  5  8 21  3  3 16]
 [22  3  0  0  2 10 33  8  4  3  9]
 [24  2  3  0  3  5  3 14  0 20  9]
 [18  0  2  2  1 10 18 10  2  3 19]]
Accurac